In [12]:
import scanpy as sc
from cellflow.model import CellFlow
import requests
import pandas as pd
import anndata
import pickle
import numpy as np

In [2]:
# Use vcc benchmark to evaluate cellflow
# load model
model = "cf-128-256"
with open(f"models/{model}_CellFlow.pkl", "rb") as f:
    cf = pickle.load(f)
cf

In [20]:
# load evaluation dataset
adata = sc.read_h5ad("../vcc_sample.h5ad")
adata.obs['control'] = [(lambda x: True if x == "non-targeting" else False)(x) for x in adata.obs['target_gene']]

In [15]:
with open("result.pkl", "rb") as f:
    res = pickle.load(f)

In [16]:
for k in res.keys():
    res[k] = np.sum(res[k],axis = 0)
res

{('ACVR1B',
  'Flex_1_10'): array([-2.1314578, 11.54824  , -6.2045865, ..., -1.4024506, 37.870556 ,
        21.253637 ], shape=(18080,), dtype=float32),
 ('AKT2',
  'Flex_1_10'): array([-2.2784128, 11.703699 , -6.2390676, ..., -1.3739402, 37.90741  ,
        21.204565 ], shape=(18080,), dtype=float32),
 ('BIRC2',
  'Flex_1_10'): array([-2.11576  , 11.575914 , -6.0469007, ..., -1.4226235, 37.842266 ,
        21.233704 ], shape=(18080,), dtype=float32),
 ('CLDN6',
  'Flex_1_10'): array([-2.2154608, 11.593581 , -6.222178 , ..., -1.3817397, 37.82464  ,
        21.341343 ], shape=(18080,), dtype=float32),
 ('HMGN1',
  'Flex_1_10'): array([-1.8522832, 11.606814 , -6.1460853, ..., -1.4817802, 37.786625 ,
        21.416567 ], shape=(18080,), dtype=float32),
 ('IGF2R',
  'Flex_1_10'): array([-1.9916604, 11.651579 , -6.0800247, ..., -1.350852 , 37.804886 ,
        21.286856 ], shape=(18080,), dtype=float32),
 ('INSIG1',
  'Flex_1_10'): array([-1.9329021, 11.55033  , -6.0384765, ..., -1.4020823, 

In [18]:
cf.adata.obs['target_gene'].unique()

['non-targeting', 'PAGR1', 'TCF3', 'NCK2', 'GSK3B', ..., 'SLC39A6', 'BRD9', 'SNCA', 'SUPT4H1', 'KIF20A']
Length: 147
Categories (147, object): ['ACAT2', 'ACVR1B', 'AKT2', 'ANTXR1', ..., 'ZNF562', 'ZNF593', 'ZNF714', 'non-targeting']

In [19]:
adata.obs['target_gene'].unique()

['non-targeting']
Categories (1, object): ['non-targeting']

In [23]:
with open("subsample_gene_embedding.pkl", "rb") as f:
    embedding = pickle.load(f)
cf.adata.uns['embedding'] = embedding

In [27]:
test_genes = [x[0] for x in res.keys()]

In [ ]:
l

13

In [28]:
eval_set = adata[adata.obs['target_gene'].isin(test_genes)]

In [32]:
eval_set.X

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 83775793 stored elements and shape (9557, 18080)>

In [34]:
x = pd.DataFrame(eval_set.X.toarray(), index = eval_set.obs.index, columns = eval_set.var_names)
x['target_gene'] = eval_set.obs['target_gene']
x

,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,ISG15,AGRN,RNF223,C1orf159,...,MT-CO2,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,target_gene
TTCCATTGTTGGTTTAAACGCCGA-Flex_2_15,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,...,10.0,3.0,2.0,10.0,0.0,7.0,0.0,0.0,2.0,HMGN1
GAGGTTTAGGTTCGAGAGTGAGTG-Flex_2_10,0.0,1.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,3.0,8.0,7.0,19.0,0.0,13.0,0.0,2.0,3.0,IGF2R
AGGTGGGAGGATGTTAATCATGTG-Flex_2_14,0.0,3.0,1.0,0.0,0.0,0.0,2.0,9.0,0.0,0.0,...,19.0,29.0,15.0,30.0,1.0,40.0,0.0,2.0,3.0,KDR
TCTTCATGTGCGATAAACTACTCA-Flex_3_12,0.0,3.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,...,1.0,1.0,1.0,3.0,0.0,2.0,0.0,0.0,1.0,ACVR1B
GGGTTGGGTCATCCTCATTCGGTT-Flex_1_16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,3.0,3.0,5.0,5.0,0.0,6.0,0.0,1.0,2.0,KAT2A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AGCATCCTCATGGTCCATACGTCA-Flex_2_13,0.0,0.0,0.0,0.0,0.0,0.0,2.0,7.0,0.0,0.0,...,9.0,6.0,3.0,3.0,0.0,5.0,0.0,2.0,1.0,HMGN1
AGCGAACGTAACTACCAACGGGAA-Flex_2_02,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0,...,11.0,10.0,6.0,16.0,0.0,13.0,1.0,1.0,3.0,IGF2R
CCTATGGGTTACCGTGAAGTAGAG-Flex_2_07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,16.0,20.0,11.0,10.0,0.0,16.0,0.0,4.0,3.0,IGF2R
CCAAACCTCTTTAGCGAGCTGTGA-Flex_3_08,0.0,4.0,0.0,0.0,0.0,0.0,4.0,16.0,0.0,0.0,...,4.0,6.0,0.0,5.0,0.0,7.0,0.0,2.0,0.0,KDR


In [57]:
# ground truth
true = x.groupby("target_gene").median().to_numpy()

/var/folders/9t/47g679_n6zvcr33prvzd5qzh0000gn/T/ipykernel_52015/1746648647.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  true = x.groupby("target_gene").median().to_numpy()


In [52]:
pred = np.array(list(res.values()))

In [ ]:
# MAE
abs(true-pred).mean()

np.float32(38.662685)